In [2]:
from pathlib import Path
import torch
import json
from sentence_transformers import SentenceTransformer
import numpy as np

In [4]:
model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
model

You try to use a model that was created with version 3.0.0.dev0, however, your version is 3.0.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [5]:
p_nlq_val_json = Path('/data/gunsbrother/prjs/ltvu/llms/GroundVQA/data/unified/annotations.NLQ_val.json')
nlq_val_data = json.load(p_nlq_val_json.open())
clip_uids = set(entry['video_id'] for entry in nlq_val_data)
clip_uid = np.random.choice(list(clip_uids))
questions, gts = [], []
for entry in nlq_val_data:
	if entry['video_id'] == clip_uid:
		questions.append(entry['question'])
		gts.append((entry['clip_start_sec'], entry['clip_end_sec']))
gts = np.array(gts)

print(f'Clip UID: {clip_uid}')

p_caps_dir = Path('/data/gunsbrother/prjs/ltvu/llms/LLaVA/results/egonlq/llava-v1.6-34b/global')
p_cap = p_caps_dir / f'{clip_uid}.json'
cap_data = json.load(p_cap.open())['answers']
caps = [cap[-1] for cap in cap_data]

Clip UID: a769264b-e62e-4515-be98-a5828a533e45


In [18]:
z_caps = model.encode(caps, convert_to_tensor=True, convert_to_numpy=False)
z_qs = model.encode(questions, prompt='query: ', convert_to_tensor=True, convert_to_numpy=False)
z_caps.shape, z_qs.shape

(torch.Size([44, 768]), torch.Size([12, 768]))

In [20]:
z_qs

tensor([[ 0.3585, -0.4076, -0.3300,  ..., -0.0693, -0.0899, -0.1899],
        [ 0.3008, -0.2176, -0.2229,  ...,  0.2494, -0.3454, -0.0568],
        [ 0.3585, -0.4076, -0.3300,  ..., -0.0693, -0.0899, -0.1899],
        ...,
        [-0.0193, -0.4487, -0.2293,  ..., -0.0203, -0.0980,  0.0067],
        [-0.0335, -0.4203, -0.1321,  ..., -0.1301, -0.3933, -0.0419],
        [ 0.0033, -0.2894, -0.2347,  ..., -0.0927, -0.2202, -0.0320]])

In [22]:
from datasets import Dataset

sentence1_list = ["My first sentence", "Another pair"]
sentence2_list = ["My second sentence", "Unrelated sentence"]
labels_list = [0.8, 0.3]
train_dataset = Dataset.from_dict({
    "sentence1": sentence1_list,
    "sentence2": sentence2_list,
    "label": labels_list,
})

In [24]:
train_dataset[1]

{'sentence1': 'Another pair', 'sentence2': 'Unrelated sentence', 'label': 0.3}